## BC Raw Log Exports

Data taken from [Forest - BC Govt](https://www2.gov.bc.ca/gov/content/data/statistics/business-industry-trade/industry/forestry)


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

In [38]:
raw_data = pd.read_csv("raw_data/BCLogExports_MinFor_180112.csv",header=1,thousands=",").dropna(how='any')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7735 entries, 0 to 7734
Data columns (total 6 columns):
Year                   7735 non-null object
Month                  7735 non-null object
Commodity              7735 non-null object
Destination Country    7735 non-null object
Value - Cdn $          7735 non-null float64
Quantity - m3          7735 non-null float64
dtypes: float64(2), object(4)
memory usage: 423.0+ KB


In [39]:
raw_data.columns = ["Year","Month","Commodity",'Destination','Value','Quantity']

In [40]:
dfMonths = pd.DataFrame({'Month':['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],'MonthIndex':range(12)})
data = pd.merge(raw_data,dfMonths,on="Month")

In [41]:
data.groupby(['Year','Destination'])['Value','Quantity'].sum()

Value   Quantity
Year Destination                               
1988 Albania                290450.0     3449.0
     Australia              106882.0      604.0
     Austria                 65730.0      472.0
     Germany                143428.0      506.0
     Greece                  16943.0      217.0
     Hong Kong               70936.0     1264.0
     India                   87302.0     1089.0
     Japan               220960208.0  1939333.0
     Mainland China       16459712.0   312211.0
     New Zealand              9830.0       77.0
     North Korea             42327.0      597.0
     Pakistan                10112.0       18.0
     South Korea          10709724.0   200890.0
     Switzerland            242061.0     4588.0
     Taiwan                 379638.0      947.0
     Turkey                 598083.0    15941.0
     United Kingdom          22336.0      276.0
     United States        44716632.0   700642.0
1989 Finland                  9600.0       87.0
     France                  47317.0      343.0
     Germany                107713.0     1041.0
     Israel                 344038.0     4734.0
     Italy                   38549.0     1572.0
     Japan               139232253.0  1186930.0
     Mainland China        4891029.0    84461.0
     Singapore               78936.0     1721.0
     South Korea           7879144.0   135376.0
     Taiwan                  99626.0      540.0
     United States        30680113.0   551251.0
1990 Germany                150819.0      609.0
...                              ...        ...
2015 Japan               139749449.0  1096478.0
     Mainland China      334470968.0  2904323.0
     New Zealand             47826.0       42.0
     Pakistan                90225.0      370.0
     South Korea         120151309.0   941770.0
     Taiwan                5934406.0    20888.0
     United Kingdom          59687.0       83.0
     United States        64805316.0   601081.0
2016 India                   10767.0       50.0
     Japan               190929902.0  1434530.0
     Mainland China      428602442.0  3685817.0
     New Zealand             39003.0       18.0
     Pakistan                50822.0      199.0
     South Korea          97804746.0   721123.0
     Taiwan                4801026.0    15136.0
     Thailand                25400.0       23.0
     United States        48250609.0   425825.0
     Vietnam                  6228.0       50.0
2017 Hong Kong              110547.0      787.0
     Japan               156668689.0  1126717.0
     Mainland China      448059760.0  3460094.0
     Netherlands            225135.0     1047.0
     New Zealand              1100.0        5.0
     Pakistan                12415.0       66.0
     Philippines           1640347.0     4881.0
     South Korea          97172619.0   633645.0
     Spain                  120000.0      268.0
     St Pierre-Miquelin       1012.0        1.0
     Taiwan                2037489.0     8360.0
     United States        41351630.0   348540.0

[341 rows x 2 columns]